In [11]:
import os
import sqlite3
import xml.etree.ElementTree as ET
import pandas as pd

In [12]:
# Function to extract speaker information
def get_speaker_info(speaker):
    name = speaker.get("naam")
    party = speaker.get("partij")
    return name, party

# Creating SQLite database
db_file = "speeches.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Creating table in the database
cursor.execute('''CREATE TABLE IF NOT EXISTS speeches
                  (file_name TEXT,date TEXT, speaker_name TEXT, party TEXT, category TEXT, title TEXT, speech TEXT)''')

conn.commit()

def process_files(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.xml'):
                    print(file)
                    file_path = os.path.join(root, file)
                    tree = ET.parse(file_path)
                    root_elem = tree.getroot()
                    # Extracting date
                    try:
                        date_element = root_elem.find(".//meta[@name='OVERHEIDop.datumVergadering']")
                        date = date_element.get("content") if date_element is not None else ""
                    except Exception as e:
                        print(f"Error extracting date from XML file {file_path}: {str(e)}")
                        date = ""

                    # Extracting category
                    try:
                        category_element = root_elem.find(".//meta[@name='OVERHEID.category']")
                        category = category_element.get("content") if category_element is not None else ""
                    except Exception as e:
                        print(f"Error extracting category from XML file {file_path}: {str(e)}")
                        category = ""
                    
                    # Extracting title
                    try:
                        title_element = root_elem.find(".//meta[@name='DC.title']")
                        title = title_element.get("content") if title_element is not None else ""
                    except Exception as e:
                        print(f"Error extracting title from XML file {file_path}: {str(e)}")
                        title = ""
                    # Get file name
                    file_name = file
                    speeches = root_elem.findall(".//spreekbeurt")
                    for speech in speeches:
                         speaker_name, party = get_speaker_info(speech)
                         if not speaker_name:
                            try:
                                speaker_name=speech.find("spreker/naam/achternaam").text.strip() #if speech_content_element is not None else ""  
                            except:
                                speaker_name=speech.find("spreker").text.strip() #if speech_content_element is not None else ""  
                         speech_content_element = speech.find("tekst/al")
                         speech_content = speech_content_element.text.strip() if speech_content_element is not None else ""
                         if not speech_content:
                            tekst_elements = speech.findall("tekst")
                            
                            speech_content_elements = []
                            for tekst_element in tekst_elements:
                                speech_content_groep_elements = tekst_element.findall("al-groep")
                                for speech_content_groep_element in speech_content_groep_elements:
                                    al_elements = speech_content_groep_element.findall("al")
                                    # for al_element in al_elements:
                                    #     speech_content_elements.append(al_element.text.strip() if al_element.text is not None else "")
                                    for al_element in al_elements:
                                        text = al_element.text.strip() if al_element.text is not None else ""
                                        tail = al_element.tail.strip() if al_element.tail is not None else ""
                                        speech_content_elements.append(text + tail)
                            speech_content = " ".join(speech_content_elements)
                            





                         if not speech_content:
                             speech_content_element = speech.find("tekst/motie/al")
                             speech_content = speech_content_element.text.strip() if speech_content_element is not None else ""



                             
                             
                         cursor.execute("INSERT INTO speeches (file_name,date, speaker_name, party, category, title, speech) VALUES (?, ?, ?, ?, ?, ?, ?)",
                                        (file_name,date, speaker_name, party, category, title, speech_content))
                         conn.commit()



# Path to the root folder with XML files
data_folder = 'xml_data'

# Call the function to process the files
process_files(data_folder)

# Close the database connection
conn.close()


2012-09-18 - h-tk-20122013-1-1.xml
2012-09-18 - h-tk-20122013-1-2.xml
2012-09-18 - h-tk-20122013-1-3.xml
2012-09-18 - h-tk-20122013-1-4.xml
2012-09-19 - h-tk-20122013-2-1.xml
2012-09-19 - h-tk-20122013-2-2.xml
2012-09-19 - h-tk-20122013-2-3.xml
2012-09-19 - h-tk-20122013-2-4.xml
2012-09-19 - h-tk-20122013-2-5.xml
2012-09-19 - h-tk-20122013-2-6.xml
2012-09-20 - h-tk-20122013-3-1.xml
2012-09-20 - h-tk-20122013-3-10.xml
2012-09-20 - h-tk-20122013-3-11.xml
2012-09-20 - h-tk-20122013-3-2.xml
2012-09-20 - h-tk-20122013-3-3.xml
2012-09-20 - h-tk-20122013-3-4.xml
2012-09-20 - h-tk-20122013-3-5.xml
2012-09-20 - h-tk-20122013-3-6.xml
2012-09-20 - h-tk-20122013-3-7.xml
2012-09-20 - h-tk-20122013-3-8.xml
2012-09-20 - h-tk-20122013-3-9.xml
2012-09-25 - h-tk-20122013-4-1.xml
2012-09-25 - h-tk-20122013-4-10.xml
2012-09-25 - h-tk-20122013-4-11.xml
2012-09-25 - h-tk-20122013-4-2.xml
2012-09-25 - h-tk-20122013-4-3.xml
2012-09-25 - h-tk-20122013-4-4.xml
2012-09-25 - h-tk-20122013-4-5.xml
2012-09-25 - h-t

In [13]:
# Connecting to the SQLite database
db_file = "speeches.db"
conn = sqlite3.connect(db_file)

# Querying data from the database
query = "SELECT * FROM speeches"

# Reading data from the database into a DataFrame
df = pd.read_sql_query(query, conn)
df.replace("", pd.NA, inplace=True)
# Closing the database connection
conn.close()

# Printing the first few rows of the DataFrame
print(df.head())


                            file_name        date       speaker_name  \
0  2012-09-18 - h-tk-20122013-1-2.xml  2012-09-18         Voorzitter   
1  2012-09-18 - h-tk-20122013-1-3.xml  2012-09-18         Voorzitter   
2  2012-09-18 - h-tk-20122013-1-3.xml  2012-09-18  Jan Kees de Jager   
3  2012-09-18 - h-tk-20122013-1-3.xml  2012-09-18  Jan Kees de Jager   
4  2012-09-18 - h-tk-20122013-1-3.xml  2012-09-18         Voorzitter   

        party               category  \
0  Voorzitter    Bestuur | Parlement   
1  Voorzitter  Financiën | Begroting   
2    Regering  Financiën | Begroting   
3    Regering  Financiën | Begroting   
4  Voorzitter  Financiën | Begroting   

                                            title  \
0                                   Mededelingen    
1  aanbieding van de begroting voor het jaar 2013   
2  aanbieding van de begroting voor het jaar 2013   
3  aanbieding van de begroting voor het jaar 2013   
4  aanbieding van de begroting voor het jaar 2013   

       